In [1]:
import multiprocessing
import sys
from pettingzoo.test import parallel_api_test
from citylearn import GridLearn
from citylearn import MyEnv
from pathlib import Path
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3 import PPO
import gym
import numpy as np
import supersuit as ss

import time

# multiprocessing.set_start_method("fork")

climate_zone = 1
data_path = Path("../citylearn/data/Climate_Zone_"+str(climate_zone))
buildings_states_actions = '../citylearn/buildings_state_action_space.json'

config = {
    "data_path":data_path,
    "climate_zone":climate_zone,
    "buildings_states_actions_file":buildings_states_actions,
    "hourly_timesteps":4,
#     "percent_rl":0.1,
    "percent_rl":1,
    "nclusters":2,
#     "max_num_houses":None
    "max_num_houses":4
}

grid = GridLearn(**config)

envs = [MyEnv(grid) for _ in range(config['nclusters'])]

print('padding action/observation spaces...')
envs = [ss.pad_action_space_v0(env) for env in envs]
envs = [ss.pad_observations_v0(env) for env in envs]

print('creating pettingzoo env...')
envs = [ss.pettingzoo_env_to_vec_env_v0(env) for env in envs]

print('stacking vec env...')
nenvs = 2
envs = [ss.concat_vec_envs_v0(env, nenvs, num_cpus=1, base_class='stable_baselines3') for env in envs]


../citylearn/buildings_state_action_space.json


/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:510: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:510: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:510: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/D

padding action/observation spaces...
creating pettingzoo env...
stacking vec env...


In [2]:
from copy import deepcopy
grid2 = deepcopy(grid)

In [3]:
grids = [grid, grid2]

In [4]:
print('setting the grid...')
for env in envs:
    for n in range(nenvs):
        env.venv.vec_envs[n].par_env.aec_env.env.env.env.env.grid = grids[n]
        env.venv.vec_envs[n].par_env.aec_env.env.env.env.env.initialize_rbc_agents()

setting the grid...


In [7]:
grid.clusters

[(['JHBY9', '24SN7'], []), (['88CD4', 'JBAQL'], [])]

In [5]:
models = [PPO(MlpPolicy, env, verbose=2, gamma=0.999, batch_size=2, n_steps=1, ent_coef=0.01, learning_rate=0.00025, vf_coef=0.5, max_grad_norm=0.5, gae_lambda=0.95, n_epochs=4, clip_range=0.2, clip_range_vf=1) for env in envs]

Using cpu device
Using cpu device


In [6]:
for _ in range(5):
    for model in models:
        model.learn(1, reset_num_timesteps=False)

calling reset...
calling reset...
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-2.2353027919733475e-05
-3.927040999653489e-05
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-2.2353027919733475e-05
-3.927040999653489e-05
--------------------------
| time/              |   |
|    fps             | 1 |
|    iterations      | 1 |
|    time_elapsed    | 2 |
|    total_timesteps | 4 |
--------------------------
calling reset...
calling reset...
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-2.257537517614822e-06
-0.0001464090289846812
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-2.257537517614822e-06
-0.0001464090289846812
-----------------------------------------
| time/                   |             |
|    fps                  | 160         |
|    iterations           | 1  

<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-6.227285892151897e-05
-0.00011935636327841757
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-6.227285892151897e-05
-0.00011935636327841757
-----------------------------------------
| time/                   |             |
|    fps                  | 249         |
|    iterations           | 1           |
|    time_elapsed         | 0           |
|    total_timesteps      | 20          |
| train/                  |             |
|    approx_kl            | 0.016405255 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    clip_range_vf        | 1           |
|    entropy_loss         | -2.84       |
|    explained_variance   | -52.2       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0328     |
|    n_updates            | 16          |
|    policy_gradient_loss | 0.0064

In [ ]:
np.array(env.venv.vec_envs[n].par_env.aec_env.env.env.env.env.grid.voltage_data).shape

In [ ]:
import matplotlib.pyplot as plt

for i in range(33):
    plt.plot(np.arange(10), np.array(grid.voltage_data)[:,i])
    
plt.savefig('voltage')

In [8]:
obss = [env.reset() for env in envs]
for _ in range(5):
    for m in range(len(models)):
        
        # get the current observation from the perspective of this team
        foo = []
        for e in range(nenvs):
            bar = list(envs[m].venv.vec_envs[e].par_env.aec_env.env.state().values())
            for i in range(len(bar)):
                while len(bar[i]) < 19:
                    bar[i] = np.append(bar[i], 0)
            foo += bar

        foo = np.vstack(foo)
        obss[m] = np.vstack(foo)

        action = models[m].predict(obss[m])[0]
        obss[m], reward, done, info = envs[m].step(action)
        print("reward",np.sum(reward))

calling reset...
calling reset...
calling reset...
calling reset...
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-6.530213343408251e-05
-0.00013072110259499504
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-6.530213343408251e-05
-0.00013072110259499504
reward -0.00039204647
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-2.257537517614822e-06
-0.0001464090289846812
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-2.257537517614822e-06
-0.0001464090289846812
reward -0.00029733314
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-6.382938840261645e-05
-0.00012403938556488064
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
-6.382938840261645e-05
-0.00012403938556488064
reward -

In [ ]:
grid.clusters

In [ ]:
from guppy import hpy

h= hpy()
h.heap()


In [ ]:
len(grid.buildings)

In [ ]:
2000/484